# Library Setup

In [ ]:
!pip install wandb
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install datasets
!pip install contractions
!pip install iterative-stratification==0.1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



     |████████████████████████████████| 1.9 MB 5.1 MB/s 




     |████████████████████████████████| 182 kB 79.8 MB/s 













     |████████████████████████████████| 166 kB 79.5 MB/s 








     |████████████████████████████████| 63 kB 903 kB/s 










     |████████████████████████████████| 166 kB 81.1 MB/s 


     |████████████████████████████████| 162 kB 79.8 MB/s 


     |████████████████████████████████| 162 kB 92.7 MB/s 


     |████████████████████████████████| 158 kB 81.4 MB/s 


     |████████████████████████████████| 157 kB 87.7 MB/s 


     |████████████████████████████████| 157 kB 55.7 MB/s 


     |████████████████████████████████| 157 kB 93.9 MB/s 


     |████████████████████████████████| 157 kB 88.3 MB/s 


     |████████████████████████████████| 157 kB 95.0 MB/s 


     |████████████████████████████████| 157 kB 79.9 MB/s 


     |████████████████████████████

In [ ]:
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

rm: cannot remove '/root/.kaggle': No such file or directory

ref                                                             title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  

--------------------------------------------------------------  ------------------------------------------  ------  -------------------  -------------  ---------  ---------------  

akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                               9KB  2022-10-06 08:55:25           9149        296  1.0              

whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset        11KB  2022-09-30 04:05:11           7296        234  1.0              

whenamancodes/students-performance-in-exams                     Students Performance in Exams                  9KB  2022-09-14 15:14:54          12756        241  1.0              

dimitryzub/walmart-coffee-li

In [ ]:
!kaggle competitions download -c feedback-prize-english-language-learning


  0% 0.00/2.80M [00:00<?, ?B/s]

100% 2.80M/2.80M [00:00<00:00, 193MB/s]


In [ ]:
!unzip feedback-prize-english-language-learning.zip -d data

Archive:  feedback-prize-english-language-learning.zip

  inflating: data/sample_submission.csv  

  inflating: data/test.csv           

  inflating: data/train.csv          


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import transformers
import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import glob
import os
import wandb
import random
import matplotlib.pyplot as plt
import numpy as np
from transformers import AdamW
from text_unidecode import unidecode
import torch
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AutoConfig
import re
from torch.nn import Module
from collections import Counter, defaultdict
from tqdm import tqdm
from torch import nn
import unicodedata
from copy import deepcopy
import sys
import gc
import contractions
from transformers import DataCollatorWithPadding
import codecs

# From this Gist: https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
   

In [ ]:
CFG = {
    "model_name": "microsoft/deberta-v3-base",
    "type": "Weighted Regression Head",
    "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],

    "max_length": 512,
    "seed": 42,
    "folds": 4,
    "lr": 2e-5, 
    "batch_size": 16,
    "epochs": 20,
    "num_warmup_steps": 0.0,
    "patience": 6,
    "grad_accum": 1,
    "pooler": "weighted",
    "weight_decay": 0.3,
    "grad_norm": 1000,
    
    "optimizer": "AdamW",
    "scheduler": "linear",
}
CFG["tokenizer"] = AutoTokenizer.from_pretrained(CFG["model_name"])

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.

  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
seed_everything(CFG["seed"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG["model_name"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#PLEASE DON'T USE MY API KEY. I'M TRUSTING YOU.

os.environ["XRT_TPU_CONFIG"] = "tpu_worker;0;10.0.0.2:8470"
os.environ['WANDB_CONSOLE'] = 'off'
os.environ['WANDB_NOTEBOOK_NAME'] = '/content/drive/MyDrive/Colab Notebooks/Kaggle/ELL Competition/Baselines/ELL Competition - DebertaV3Base Baseline'

env: "WANDB_API_KEY"="97d665030810b5d298011344ac2977f983a4014f"


In [ ]:
wandb.login()

wandb: Currently logged in as: goggins (kagglers). Use `wandb login --relogin` to force relogin


True

# Trackers

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class ModelTracker():
    def __init__(self, patience, base_path, model, path, optimizer, scheduler, mode = "maximize", metric_name = "accuracy"):
        self.patience = patience
        self.mode = mode
        self.missed = 0
        self.path = path
        self.model = model
        self.base_path = base_path
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric = float("-inf") if self.mode == "maximize" else float("inf")
        self.metric_name = metric_name

    def update(self, value, epoch):
        if self.mode == "maximize":
            if value >= self.metric:
                print(f"Validation {self.metric_name} rose from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                self.metric = value
                
                torch.save({
                    "epoch": epoch, 
                    "model_state_dict": self.model.state_dict(), 
                    "optimizer_state_dict": self.optimizer.state_dict(),
                    "scheduler": self.scheduler.state_dict()
                }, f"{self.base_path}/{self.path}")

                print(f"Saved to model to {self.base_path}/{self.path}!")

                self.missed = 0

            else:
                print(f"Validation {self.metric_name} fell from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                print(f"Model did not improve on epoch {epoch}")
                self.missed += 1
        else:
            if value <= self.metric:
                print(f"Validation {self.metric_name} fell from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                self.metric = value
                
                torch.save({
                    "epoch": epoch, 
                    "model_state_dict": self.model.state_dict(), 
                    "optimizer_state_dict": self.optimizer.state_dict(),
                    "loss": self.metric,
                    "scheduler": self.scheduler.state_dict()
                }, f"{self.base_path}/{self.path}")

                self.missed = 0

                print(f"Saved to model to {self.base_path}/{self.path}!")

            else:
                print(f"Validation {self.metric_name} rose from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                print(f"Model did not improve on epoch {epoch}")
                self.missed += 1

    def get_full_path(self):
        return f"{self.base_path}/{self.path}"
        
    def check_improvement(self):
        return self.missed < self.patience

# Model Definition

In [ ]:
class WeightedLayerPooling(torch.nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average
    

In [ ]:
#OBVIOUSLY, CHANGE THIS AS YOU NEED. USE SELF.LOG FOR ALL IMPORTANT METRICS
class Model(nn.Module):
    def __init__(self, config, vocab_length, data_loader_len):
        super(Model, self).__init__()
        self.config = config
        self.vocab_length = vocab_length
        self.base_model = AutoModel.from_pretrained(self.config['model_name'], output_hidden_states = True)  
        self.base_model.resize_token_embeddings(vocab_length)

        if config["pooler"] == "weighted":
            self.pooler = WeightedLayerPooling(self.base_model.config.num_hidden_layers, layer_start = self.config["layer_start"])  
            self._init_weights(self.pooler.layer_weights)

        self.fc = nn.Linear(self.base_model.config.hidden_size, 6)

        self._init_weights(self.fc)
        self.data_loader_len = data_loader_len

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):

        if self.config["pooler"] == "weighted":
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["hidden_states"]

            x = torch.stack(x)
            cls_embeddings = self.pooler(x)[:, 0]

            return cls_embeddings
            
        else:
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["last_hidden_state"]

            return x[:, 0, :]

            

    def forward(self, inputs):
        
        features = self.feature(inputs)

        return self.fc(features)

In [ ]:
def configure_optimizers(config, model):

    if config["optimizer"] == "AdamW":
        optimizer = AdamW(model.parameters(), weight_decay = config["weight_decay"], lr=config["lr"], correct_bias = True)
        
    else:
        optimizer = AdamW(model.parameters(), weight_decay = config["weight_decay"], lr=config["lr"], correct_bias = True)

    if config["scheduler"] == "cosine":
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = config["num_warmup_steps"], num_training_steps = model.data_loader_len * config["epochs"] // config["grad_accum"])

    elif config['scheduler'] == "one_cycle":
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = config["lr"], pct_start = config["pct_start"], total_steps = model.data_loader_len * config["epochs"] // config["grad_accum"])
        
    elif config['scheduler'] == "linear":
        scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps = model.data_loader_len * config["epochs"] // config["grad_accum"] * 0.1, num_training_steps = model.data_loader_len * config["epochs"] // config["grad_accum"])
        
    else:
        scheduler = None
        
    
    return optimizer, scheduler

# Dataset Classes

In [ ]:
class Data(Dataset):
    def __init__(self, df, config, indices, base_path, special_tokens = None):
        self.df = df
        self.esc_chars = ['\"', "\\", "\n", "\r", "\t", "\b", "\f", "\v", ":)", ";)", ":(", "uwu", "owo", "xd", ":3", ":-)", ":D", ">:(", "\xa0", "\x92", "\x93", "\x91", "\x94", "\x97", "x\B4", "\x96", "\x82", "\x84"]
        self.data = self.df.iloc[indices]
        self.data["full_text"] = self.data["full_text"].apply(lambda text: self.resolve_encodings_and_normalize(text))

        codecs.register_error("replace_encoding_with_utf8", self.replace_encoding_with_utf8)
        codecs.register_error("replace_decoding_with_cp1252", self.replace_decoding_with_cp1252)

    def replace_encoding_with_utf8(self, error):
        return error.object[error.start : error.end].encode("utf-8"), error.end


    def replace_decoding_with_cp1252(self, error):
        return error.object[error.start : error.end].decode("cp1252"), error.end


    def resolve_encodings_and_normalize(self, text: str) -> str:
        text = (
            text.encode("raw_unicode_escape")
            .decode("utf-8", errors="replace_decoding_with_cp1252")
            .encode("cp1252", errors="replace_encoding_with_utf8")
            .decode("utf-8", errors="replace_decoding_with_cp1252")
        )
        
        text = unidecode(text)
        
        return self.remove_esc_chars(text)

    def remove_esc_chars(self, text):
        txt = deepcopy(text)
        for char in self.esc_chars:
            if char == '\"':
                txt = txt.replace(char, '"')
            elif char == "\x92" or char == "\x91" or char == "\xB4":
                txt = txt.replace(char, "'")
            elif char == "\0x93" or char == "\0x94":
                txt = txt.replace(char, '"')
            elif char == "\0x97" or char == "\0x96":
                txt = txt.replace(char, '-')
            elif char == "\0x82" or char == "\0x84":
                txt = txt.replace(char, ',')
            else:
                txt = txt.replace(char, ' ')
        return txt

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]["full_text"], self.data.iloc[idx][2:].values.astype(float)

In [ ]:
def collate_dynamic_padding(batch):
    # Each element in the batch is in the form [(text, [labels])]
    # Converting batch to [[text...], [labels...]]
    data = np.array(batch).T
    texts = data[0]
    # Dynamic Padding tokenization
    sentences = tokenizer(list(texts), padding=True, max_length = CFG["max_length"], truncation = True, return_token_type_ids = False, return_tensors="pt")
    labels = torch.from_numpy(np.vstack(data[1]).astype(float))

    return sentences, labels

In [ ]:
#CHANGE AS NEEDED. MOST OF THE TIME, PYTORCH'S DEFAULT COLLATOR IS ENOUGH.
class DataModule():

    def __init__(self, config, train, val, collate_fn):
        self.config = config
        self.train, self.val = train, val
        self.collate_fn = collate_fn
        
    def train_dataloader(self):
        train_loader = DataLoader(self.train, batch_size=self.config["batch_size"], shuffle = True, collate_fn = self.collate_fn, pin_memory=True, num_workers = 8)
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(self.val, batch_size = self.config["batch_size"], collate_fn = self.collate_fn, pin_memory=True, num_workers = 8)      
        return val_loader

# Training

In [ ]:
def construct_mcrmse():
    def mcrmse_loss(outputs, targets):
        colwise_mse = torch.mean(torch.square(targets - outputs), dim=0)
        loss = torch.mean(torch.sqrt(colwise_mse), dim=0)
        return loss
    return mcrmse_loss

In [ ]:
def train_fn(config, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()

    losses = AverageMeter()

    pbar = tqdm(train_loader, desc = f"Training Loop Epoch: {epoch}")

    scaler = torch.cuda.amp.GradScaler()

    latest_avg = None
        
    grad_norm = 0.0

    latest_acc = 0.0

    for batch_idx, batch in enumerate(pbar):

        inputs, labels = batch

        for k, v in inputs.items():
            inputs[k] = v.to(device)

        labels = labels.to(device).to(torch.float16)

        batch_size = labels.size(0)

        #First Train
        
        with torch.cuda.amp.autocast():
            y_hat = model(inputs)

            train_loss = criterion(y_hat, labels)

            scaled_loss = train_loss / config["grad_accum"]
        
        losses.update(train_loss.item(), batch_size)
        
        scaler.scale(scaled_loss).backward()
        

        if ((batch_idx + 1) % config["grad_accum"] == 0) or (batch_idx + 1 == model.data_loader_len):

            scaler.unscale_(optimizer)

            # Since the gradients of optimizer's assigned params are unscaled, clips as usual:
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config["grad_norm"])

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad()

            if not scheduler is None:

                scheduler.step()

                for i, lr in enumerate(scheduler.get_last_lr()):
                    wandb.log({f"Layer {i} Learning Rate": lr})

            latest_avg = f"{losses.avg:.4f}"

        text = f"Epoch: {epoch} | Training Loss_avg: {latest_avg} | Training Loss_step: {losses.val:.4f} | Learning Rate: {scheduler.get_last_lr()[0]:.4f} | Grad: {grad_norm:.4f}" if not scheduler is None else f"Epoch: {epoch} | Training Loss_avg: {latest_avg} | Training Loss_step: {losses.val:.4f} | Grad: {grad_norm:.4f}"

        pbar.set_postfix_str(text)

        pbar.refresh()

        wandb.log({f"Training Loss_step": losses.val})

    wandb.log({f"Training Loss Epoch": losses.avg})

    print(f"Training Loss: {losses.avg}")

    return losses.avg

In [ ]:
def valid_fn(valid_loader, model, criterion, device, epoch):
    model.eval()
    
    losses = AverageMeter()

    #A MANUAL LOOP IS NEEDED HERE SINCE TRAINER FUNCTIONS DON'T GIVE YOU ACCESS TO MODEL PREDICTIONS

    pbar = tqdm(valid_loader, desc = f"Validation Loop Epoch: {epoch}")
    for batch_idx, batch in enumerate(pbar):

        inputs, labels = batch

        for k, v in inputs.items():
            inputs[k] = v.to(device)

        labels = labels.to(device).to(torch.float16)

        batch_size = labels.size(0)

        model = model.to(device)
        
        with torch.no_grad():
            preds = model(inputs)

        val_loss = criterion(preds, labels).item()

        losses.update(val_loss, batch_size)

        pbar.set_postfix_str(f"Epoch: {epoch} | Validation Loss_avg: {losses.avg:.4f}")

    wandb.log({f"Validation Loss Epoch": losses.avg})

    print({f"Validation Loss Epoch": losses.avg})

    return losses.avg

In [ ]:
def test_fn(test_loader, model, criterion, device, checkpoint, class_names):
    with torch.no_grad():
        losses = AverageMeter()

        # PUT YOUR CUSTOM GRAPHS HERE
        
        # LOAD BEST MODEL CHECKPOINT

        saved = torch.load(checkpoint)
        model.load_state_dict(saved["model_state_dict"])
        model.eval()

        val_loss = torch.tensor([0]).to(device).to(torch.float16)

        #A MANUAL LOOP IS NEEDED HERE SINCE TRAINER FUNCTIONS DON'T GIVE YOU ACCESS TO MODEL PREDICTIONS

        pbar = tqdm(test_loader, desc = f"Getting Test Predictions")
        for batch_idx, batch in enumerate(pbar):

            inputs, labels = batch

            for k, v in inputs.items():
                inputs[k] = v.to(device)

            labels = labels.to(device).to(torch.float16)

            batch_size = labels.size(0)

            model = model.to(device)
            
            y_hat = model(inputs)

            val_loss = criterion(y_hat, labels)

            losses.update(val_loss, batch_size)

            #LOGGING SPECIFIC THINGS HERE #############################

    
        #LOGGING OOF PERFORMANCE
        wandb.log({f"Validation MCRSE": losses.avg})

        print(f"Validation MCRSE: {losses.avg}")

        return losses.avg

In [ ]:
%wandb
def train_loop(data_collator, config, device, weights=None, base_path = "./"):
    data = pd.read_csv(f"{base_path}/data/train.csv")
    mlskf = MultilabelStratifiedKFold(n_splits=config["folds"], shuffle=True, random_state=config["seed"])
    
    for fold, (train_index, val_index) in enumerate(mlskf.split(data, data[config["targets"]])):
        seed_everything(config["seed"])

        train = Data(data, config, train_index, base_path)
        val = Data(data, config, val_index, base_path)

        wandb.init(project="ELL", entity = "kagglers", group = config["type"], config = config, job_type = f"{config['model_name']}", save_code = True, reinit = True, name = f"Fold {fold}")

        criterion = construct_mcrmse()

        validation_criterion = construct_mcrmse()

        dataset = DataModule(config, train, val, data_collator)

        train_loader = dataset.train_dataloader()

        val_loader = dataset.val_dataloader()

        model = Model(config, len(config["tokenizer"]), len(train_loader))

        model = model.to(device)

        optimizer, scheduler = configure_optimizers(config, model)

        tracker = ModelTracker(config["patience"], base_path, model, f"fold-{fold}.pt", optimizer, scheduler, mode = "minimize", metric_name = "MCRMSE")

        for epoch in range(config["epochs"]):

            train_loss = train_fn(config, train_loader, model, criterion, optimizer, epoch, scheduler, device)

            val_loss = valid_fn(val_loader, model, validation_criterion, device, epoch)

            tracker.update(val_loss, epoch)

            if not tracker.check_improvement():
                print(f"Stopping the model at epoch {epoch} since the model did not improve!")
                break

        checkpoint = tracker.get_full_path()

        test_fn(val_loader, model, validation_criterion, device, checkpoint, config["targets"])

        wandb.save(checkpoint)

        del dataset, model

        gc.collect()

        torch.cuda.empty_cache()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")